##Attribution and license

Portions of this page are modifications based on work created and shared by Google and used according to terms described in the Creative Commons 4.0 Attribution License. Code samples are licensed under the Apache 2.0 License. For the original page refer to
 https://www.tensorflow.org/tutorials/generative/pix2pix

##Import and install required libraries

In [ ]:
import tensorflow as tf
import os
import time
from matplotlib import pyplot as plt
from IPython import display
import numpy as np
from nibabel import load as load_nii
import nibabel as nib
from skimage.transform import resize
from google.colab import drive

In [ ]:
!pip install -U tensorboard
!nvidia-smi
drive.mount('/content/drive')

## Loading training data
*Both MPRAGE and MP2RAGE images need to be previously skull-stripped

*All the 2D slices of each subject are loaded in a TF dataset

In [ ]:
def load_data(path):
    m_image = load_nii((path+'mprage_sk.nii.gz'),mmap=True)
    image = np.float32(m_image.get_fdata())
    m_gt = load_nii((path+'mp2rage_sk.nii.gz'),mmap=True)
    gt = np.float32(m_gt.get_fdata())

    ind = (np.where(image>0))
    x2 = (np.max(ind[0]))
    x1 = (np.min(ind[0]))
    y2 = (np.max(ind[1]))
    y1 = (np.min(ind[1]))
    z2 = (np.max(ind[2]))
    z1 = (np.min(ind[2]))

    z_dim = np.shape(np.squeeze(image))[2]
    n_images = (x2-x1)+(y2-y1)+(z2-z1)
    inp = np.zeros([n_images,150,150,1])
    t = np.zeros([n_images,150,150,1])

    gt[(gt>3400)]= 2000
    gt[(gt>2500) & (image<150)]= 2000

    image -= np.mean(image)
    image /= np.std(image)
    gt -= np.mean(gt)
    gt /= np.std(gt)

    for i in range(z1,z2):

        image_one = image[x1:(x1+150),y1:(y1+150), i]
        gt_one = gt[x1:(x1+150),y1:(y1+150), i]
        image_one=np.expand_dims(image_one,axis=2)
        gt_one=np.expand_dims(gt_one,axis=2)
        inp[i-z1,:,:,:]=image_one
        t[i-z1,:,:,:]=gt_one

    for i in range(y1,y2):
        image_one = np.squeeze(image[x1:(x1+150),i,:150])
        gt_one = np.squeeze(gt[x1:(x1+150),i,:150])
        image_one=np.expand_dims(image_one,axis=2)
        gt_one=np.expand_dims(gt_one,axis=2)
        inp[(z2-z1)+i-y1,:240,:z_dim,:]=image_one
        t[(z2-z1)+i-y1,:240,:z_dim,:]=gt_one
        
    for i in range(x1,x2):

        image_one = np.squeeze(image[i,y1:(y1+150),:150])
        gt_one = np.squeeze(gt[i,y1:(y1+150),:150])
        image_one=np.expand_dims(image_one,axis=2)
        gt_one=np.expand_dims(gt_one,axis=2)
        inp[(z2-z1+y2-y1)+i-x1,:256,:z_dim,:]=image_one
        t[(z2-z1+y2-y1)+i-x1,:256,:z_dim,:]=gt_one

    return inp,t

The variable path refers to the folder containing the training dataset. Each subject needs a subfolder named 1 to N, where N is the total number of training subjects.

In [ ]:
path = '../Data/' 
N= 20

x,y =load_data(path+str(1)+'/')
for i in range (2,N):
  x2,y2 =load_data(path+str(i)+'/')
  assert x2.shape[0] == y2.shape[0]
  x = np.concatenate((x,x2))
  y = np.concatenate((y,y2))
print(np.shape(x))  

Optional: applying bias field data augmentation.

In [ ]:
def bias_field(image):
  image=np.squeeze(image)
  
  r = np.random.random(1)[0]
  p = np.random.random(1)[0]
  image2=np.zeros_like(image)

  if p<0.25:
    for i in range(0,image.shape[0]):
      for k in range(0,image.shape[1]):
        image2[i,k]=image[i,k]+(i*r*0.001)
  if p>0.25 and p<0.50:
    for i in range(0,image.shape[0]):
      for k in range(0,image.shape[1]):
        image2[i,k]=image[i,k]+((image.shape[0]-i)*r*0.001)
  if p>0.50 and p<0.75:
    for i in range(0,image.shape[1]):
      for k in range(0,image.shape[0]):
        image2[i,k]=image[i,k]+(i*r*0.001)
  if p>0.75:
    for i in range(0,image.shape[1]):
      for k in range(0,image.shape[0]):
        image2[i,k]=image[i,k]+((image.shape[1]-i)*r*0.001)         

  image2=np.expand_dims(image2,0)
  image2=np.expand_dims(image2,-1)

  return image2

In [ ]:
def crop(image,label):
  
  p = tf.random.uniform([1])
  if p<0.5:
    image = tf.image.random_crop(image, [1,128,128,1], seed=1)
    label = tf.image.random_crop(label, [1,128,128,1], seed=1)
    image= tf.image.random_flip_up_down(image,seed=1)
    label= tf.image.random_flip_up_down(label,seed=1)

  if p>0.5:
    image = tf.image.random_crop(image, [1,64,64,1], seed=1)
    label = tf.image.random_crop(label, [1,64,64,1], seed=1)
    image= tf.image.resize(image,[128,128])
    label= tf.image.resize(label,[128,128])
    image= tf.image.random_flip_left_right(image, seed=1)
    label= tf.image.random_flip_left_right(label, seed=1)

  return image,label

In [ ]:
BUFFER_SIZE = np.shape(x)[0]
BATCH_SIZE = 1
IMG_WIDTH = 150
IMG_HEIGHT = 150

train_dataset = tf.data.Dataset.from_tensor_slices((x.astype(np.float32), y.astype(np.float32)))
train_dataset= train_dataset.shuffle(BUFFER_SIZE,reshuffle_each_iteration=True)
train_dataset= train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.map(crop)

Plotting 10 examples of training images loaded

In [ ]:
def plot_images(test_input, tar):
  plt.figure(figsize=(15,15))
  display_list = [np.squeeze(test_input[0]), np.squeeze(tar[0]) ]
  title = ['Input Image', 'Ground Truth']

  for i in range(2):
    plt.subplot(1, 5, i+1)
    plt.title(title[i])
    plt.imshow(display_list[i][:,:],cmap='gray')
    plt.axis('off')
  plt.show()

In [ ]:
for example_input, example_target in train_dataset.take(10):
  plot_images(example_input,example_target)

In [ ]:
del x,y

## Build the Generator
  * The architecture of the generator is a modified U-Net wtih residual blocks.


In [ ]:
OUTPUT_CHANNELS = 1

In [ ]:
def res_block(x,filters,filters2, size, apply_batchnorm=False):
  initializer = tf.random_normal_initializer(0., 0.02)
  conv1 = tf.keras.layers.Conv2D(filters, size, strides=1, padding='same',
                             kernel_initializer=initializer, use_bias=False) (x)
  if apply_batchnorm:
    conv1= tf.keras.layers.BatchNormalization() (conv1)
  conv1= tf.keras.layers.LeakyReLU()(conv1) 
  
  conv2 = tf.keras.layers.Conv2D(filters, size, strides=1, padding='same',
                             kernel_initializer=initializer, use_bias=False) (conv1)
  if apply_batchnorm:
    conv2= tf.keras.layers.BatchNormalization() (conv2)
  
  conv2=tf.keras.layers.add([x,conv2])
  conv2= tf.keras.layers.LeakyReLU()(conv2)

  return conv2

In [ ]:
def res_block_down(x,filters,filters2, size, apply_batchnorm=False):
  initializer = tf.random_normal_initializer(0., 0.02)
  conv1 = tf.keras.layers.Conv2D(filters, size, strides=1, padding='same',
                             kernel_initializer=initializer, use_bias=False) (x)
  if apply_batchnorm:
    conv1= tf.keras.layers.BatchNormalization() (conv1)
  conv1= tf.keras.layers.LeakyReLU()(conv1) 
  
  conv2 = tf.keras.layers.Conv2D(filters, size, strides=1, padding='same',
                             kernel_initializer=initializer, use_bias=False) (conv1)
  if apply_batchnorm:
    conv2= tf.keras.layers.BatchNormalization() (conv2)
  
  conv2=tf.keras.layers.add([x,conv2])
  conv2= tf.keras.layers.LeakyReLU()(conv2)

  conv3 = tf.keras.layers.Conv2D(filters2, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False) (conv2)
  if apply_batchnorm:
    conv3= tf.keras.layers.BatchNormalization() (conv3)

  conv3= tf.keras.layers.LeakyReLU()(conv3)

  return conv3

In [ ]:
def res_block_up(x,x1,filters,filters2, size, apply_batchnorm=False, concat=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  if concat:
    x = tf.keras.layers.Concatenate()([x,x1])
  
  conv1 = tf.keras.layers.Conv2D(filters, size, strides=1, padding='same',
                             kernel_initializer=initializer, use_bias=False) (x)
  if apply_batchnorm:
    conv1= tf.keras.layers.BatchNormalization() (conv1)
  conv1= tf.keras.layers.LeakyReLU()(conv1) 
  
  conv2 = tf.keras.layers.Conv2D(filters, size, strides=1, padding='same',
                             kernel_initializer=initializer, use_bias=False) (conv1)
  if apply_batchnorm:
    conv2= tf.keras.layers.BatchNormalization() (conv2)
  
  conv2=tf.keras.layers.add([conv1,conv2])
  conv2= tf.keras.layers.LeakyReLU()(conv2)

  conv3 = tf.keras.layers.Conv2DTranspose(filters2, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False)(conv2)

  if apply_batchnorm:
    conv3= tf.keras.layers.BatchNormalization() (conv3)

  conv3= tf.keras.layers.LeakyReLU()(conv3)

  return conv3

In [ ]:
def Generator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inputs = tf.keras.layers.Input(shape=[128,128,OUTPUT_CHANNELS])

  down1 = res_block_down(inputs,32,64, 4) 
  down2 = res_block_down(down1,64,128, 4) 
  down3 = res_block_down(down2,128,256, 4) 
  down4 = res_block_down(down3,256,512, 4) 
  down5 = res_block_down(down4,512,512, 4) 


  tra1 = res_block(down5,512,512, 4)
  tra2 = res_block(tra1,512,512, 4)
  tra3 = res_block(tra2,512,512, 4)
  tra4 = res_block(tra3,512,512, 4)

  up7 = res_block_up(tra4,tra4,512,256, 4, False,False)
  up3 = res_block_up(up7,down4,256,128, 4, False)
  up2 = res_block_up(up3,down3,128,64 ,4)
  up1 = res_block_up(up2,down2,64,32, 4)
  up0 = res_block_up(up1,down1,32,32, 4)

  
  last = tf.keras.layers.Conv2D(OUTPUT_CHANNELS, 3,
                                         strides=1,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') (up0)

  o = tf.math.add(inputs,last)

  return tf.keras.Model(inputs=inputs, outputs=o)

In [ ]:
generator = Generator()

In [ ]:
LAMBDA = 150
PHI= 5

In [ ]:
def generator_loss(disc_generated_output, gen_output, target,vgg_loss):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
  total_gen_loss =  gan_loss + (LAMBDA * l1_loss) + (PHI*vgg_loss)
 
  return total_gen_loss, gan_loss, l1_loss

## Build the Discriminator

In [ ]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[128, 128, 1], name='input_image')
  tar = tf.keras.layers.Input(shape=[128, 128, 1], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar]) 

  conv1 = tf.keras.layers.Conv2D(32, 4, strides=4,
                                kernel_initializer=initializer)(x) 
  conv1 = tf.keras.layers.LeakyReLU()(conv1)
  conv2 = tf.keras.layers.Conv2D(32, 4, strides=2,
                                kernel_initializer=initializer)(conv1) 
  conv2 = tf.keras.layers.LeakyReLU()(conv2)
  conv3 = tf.keras.layers.Conv2D(32, 4, strides=2,
                                kernel_initializer=initializer)(conv2) 
  conv3 = tf.keras.layers.LeakyReLU()(conv3)
  conv4 = tf.keras.layers.Conv2D(32, 4, strides=2,
                                kernel_initializer=initializer)(conv3) 
  conv4 = tf.keras.layers.LeakyReLU()(conv4)

  conv5 = tf.keras.layers.Conv2D(1, 2, strides=1,
                                kernel_initializer=initializer)(conv4)
  conv5 = tf.keras.layers.LeakyReLU()(conv5)

  flat1 = tf.keras.layers.Flatten()(conv4)
  dense1 = tf.keras.layers.Dense(32)(flat1)
  dense1 = tf.keras.layers.LeakyReLU()(dense1)
  dense2 = tf.keras.layers.Dense(16)(dense1)
  dense2 = tf.keras.layers.LeakyReLU()(dense1) 
  dense3 = tf.keras.layers.Dense(1)(dense2) 
  dense3 = tf.keras.layers.LeakyReLU()(dense3) 

  return tf.keras.Model(inputs=[inp, tar], outputs=conv5)

In [ ]:
discriminator = Discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return (total_disc_loss*0.5)

Implementation of the perceptual loss based on the second layer of the VGG16 pre-trained network. The model is donwloaded automatically from the Tf storage.

In [ ]:
vgg16_model =tf.keras.applications.VGG16(input_shape=[128,128,3],include_top=False, weights='imagenet')
vgg16_model.trainable=False

selectedLayers = [2]
selectedOutputs = [vgg16_model.layers[i].output for i in selectedLayers]
vgg16_model = tf.keras.Model(vgg16_model.inputs,selectedOutputs)
vgg16_model.summary()

In [ ]:
def perceptual_loss(out_image,out_target):
  loss=0
  for i in range (0,len(out_image)):
      loss+= tf.reduce_mean(tf.abs(tf.convert_to_tensor(out_image[i])-tf.convert_to_tensor(out_target[i])))
  return loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-5, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5, beta_1=0.5)

In [ ]:
checkpoint_dir = '/content/drive/My Drive/MRI_data/checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15,15))

  diff = np.squeeze(prediction[0]) - np.squeeze(test_input[0])
  diff2 = np.squeeze(np.squeeze(tar[0])-(np.squeeze(test_input[0])))
  display_list = [np.squeeze(test_input[0]), np.squeeze(tar[0]), np.squeeze(prediction[0]), diff, diff2 ]
  title = ['Input Image', 'Ground Truth', 'Predicted Image', 'Difference output-input', 'Real difference']


  for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i][32:96,32:96],cmap='gray')
    plt.axis('off')
  plt.show()

Visualization of the initial results when the generator is inizialized with random weights.

In [ ]:
for example_input, example_target in train_dataset.take(2):
  generate_images(generator,example_input,example_target)

## Training



In [ ]:
EPOCHS = 40

In [ ]:
import datetime
log_dir="logs/"
summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
@tf.function
def train_step(input_image, target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)
    
    out_image = vgg16_model([(tf.concat((gen_output,gen_output,gen_output),axis=-1)),gen_output],training=False)
    out_target = vgg16_model([(tf.concat((target,target,target),axis=-1)),target],training=False)
    vgg_loss = perceptual_loss(out_image,out_target)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target, vgg_loss)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))
  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))  

  with summary_writer.as_default():
    tf.summary.scalar('vgg_loss', vgg_loss, step=epoch)
    tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
    tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
    tf.summary.scalar('disc_loss', disc_loss, step=epoch)

In [ ]:
def fit(train_ds, epochs, test_ds):
  for epoch in range(epochs):
    start = time.time()

    display.clear_output(wait=True)

    for example_input, example_target in test_ds.take(3):
      generate_images(generator, example_input, example_target)
    print("Epoch: ", epoch)

    # Train
    for n, (input_image, target) in train_ds.enumerate():
      print('.', end='')
      if (n+1) % 100 == 0:
        print()
      train_step(input_image, target, epoch)
    print()

    # saving (checkpoint) the model every 10 epochs
    if (epoch + 1) % 10 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))
  checkpoint.save(file_prefix = checkpoint_prefix)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

In [ ]:
fit(train_dataset, EPOCHS, train_dataset)

## Restore a model and infer new data

In [ ]:
checkpoint.restore("../Model/ckpt-4")

In [ ]:
def load_data_test_x(path):
    m_image = load_nii((path+'mprage_sk.nii.gz'),mmap=True)
    orientation = m_image.affine
    h = m_image.header
    image = np.squeeze(np.float32(m_image.get_fdata()))
    image -= np.mean(image)
    image /= np.std(image)

    print(image.shape)
    shape= np.shape(image)
    N = np.shape(image)[0]
    y= np.shape(image)[1]
    z= np.shape(image)[2]
    output = np.zeros([N,256,256,1])

    for i in range(0,N):
        image_one = image[i,:y,:z]
        image_one=np.expand_dims(image_one,axis=2)
        output[i,:y,:z,:]=image_one

    target = np.zeros([N,256,256,1])

    for i in range(0,N):
        image_one = image[i,:y,:z]
        image_one=np.expand_dims(image_one,axis=2)
        target[i,:y,:z,:]=image_one

    return output,target,orientation,h,shape

In [ ]:
def load_data_test_y(path):
    m_image = load_nii((path+'mprage_sk.nii.gz'),mmap=True)
    orientation = m_image.affine
    image = np.squeeze(np.float32(m_image.get_fdata()))
    image -= np.mean(image)
    image /= np.std(image)

    print(image.shape)
    N = np.shape(image)[1]
    y= np.shape(image)[1]
    z= np.shape(image)[2]
    x= np.shape(image)[0]
    output = np.zeros([N,256,256,1])

    for i in range(0,N):
        image_one = image[:x,i,:z]
        image_one=np.expand_dims(image_one,axis=2)
        output[i,:x,:z,:]=image_one

    target = np.zeros([N,256,256,1])

    for i in range(0,N):
        image_one = image[:x,i,:z]
        image_one=np.expand_dims(image_one,axis=2)
        target[i,:x,:z,:]=image_one

    return output,target,orientation   

In [ ]:
def load_data_test_z(path):
    m_image = load_nii((path+'mprage_sk.nii.gz'),mmap=True)
    orientation = m_image.affine
    image = np.squeeze(np.float32(m_image.get_fdata()))
    image -= np.mean(image)
    image /= np.std(image)

    print(image.shape)
    N = np.shape(image)[2]
    y= np.shape(image)[1]
    z= np.shape(image)[2]
    x= np.shape(image)[0]
    output = np.zeros([N,256,256,1])

    for i in range(0,N):
        image_one = image[:x,:y,i]
        image_one=np.expand_dims(image_one,axis=2)
        output[i,:x,:y,:]=image_one

    m_image = load_nii((path+'mprage_sk.nii.gz'),mmap=True)
    image = np.squeeze(np.float32(m_image.get_fdata()))
    image -= np.mean(image)
    image /= np.std(image)

    
    target = np.zeros([N,256,256,1])

    for i in range(0,N):
        image_one = image[:x,:y,i]
        image_one=np.expand_dims(image_one,axis=2)
        target[i,:x,:y,:]=image_one

    return output,target,orientation   

In [ ]:
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

The variable path_test refers to the folder where the testing dataset is.

In [ ]:
path_test = '../TestData/'
for k in range (1,2):

  path= path_test+str(k)+'/'
  x,t,orien,h,shape =load_data_test_x(path)
  test_dataset = tf.data.Dataset.from_tensor_slices((x.astype(np.float32), t.astype(np.float32)))
  test_dataset= test_dataset.batch(BATCH_SIZE)
  output=np.array([])

  for inp, tar in test_dataset:
    prediction = generator(inp, training=True)
    output= np.concatenate((output,np.squeeze(prediction.numpy(),axis=0)),axis=-1) if output.size else np.squeeze(prediction.numpy(),axis=0)

  output_x=np.moveaxis(output,-1,0)

  x,t,orien =load_data_test_y(path)
  test_dataset = tf.data.Dataset.from_tensor_slices((x.astype(np.float32), t.astype(np.float32)))
  test_dataset= test_dataset.batch(BATCH_SIZE)
  output=np.array([])

  for inp, tar in test_dataset:
    prediction = generator(inp, training=True)
    output= np.concatenate((output,np.squeeze(prediction.numpy(),axis=0)),axis=-1) if output.size else np.squeeze(prediction.numpy(),axis=0)

  output_y=np.moveaxis(output,-1,1)

  x,t,orien =load_data_test_z(path)
  test_dataset = tf.data.Dataset.from_tensor_slices((x.astype(np.float32), t.astype(np.float32)))
  test_dataset= test_dataset.batch(BATCH_SIZE)
  output=np.array([])

  for inp, tar in test_dataset:
    prediction = generator(inp, training=True)
    output= np.concatenate((output,np.squeeze(prediction.numpy(),axis=0)),axis=-1) if output.size else np.squeeze(prediction.numpy(),axis=0)

  output_z=output
  x_dim=shape[0]
  y_dim=shape[1]
  z_dim=shape[2]  

  output_f = ((output_x[:x_dim,:y_dim,:z_dim]+output_y[:x_dim,:y_dim,:z_dim]+output_z[:x_dim,:y_dim,:z_dim])/3)
  nifti_out = nib.Nifti1Image(output_f, affine=orien,header=h)
  nifti_out.to_filename(path+'synt.nii.gz')
print ('Inference completed')  

The synthetic MP2RAGE images generated are saved in each testing subject's folder and named synt.nii.gz
